# TP 2 - Stations météo - À RENDRE

Dans cet exercice, vous allez appliquer les connaissances apprises lors du premier TP

Vous allez travailler avec le fichier "Integrated Surface Database Station History", qui regroupe les stations mémtéo de l'armée américaine. Ce fichier est organisé comme suit : 

* une entête de 22 lignes (qui rappelle le contenu des champs, plus des métadonnées) ;
* des champs délimités par un nombre défini de caractères, comme indiqué dans le tableau.

offset | taille | exemple | signification
-------|--------|---------|--------------
0 | 6 | 225730 | USAF = Air Force Datsav3 station number
7 | 5 | 99999 | WBAN = NCDC WBAN number
13 | 29 | LESUKONSKOE | Station name
43 | 2 | US | FIPS country ID (pays)
48 | 2 | KS | ST = State for US stations
51 | 4 | LFRO | CALL = ICAO call sign (code aéroport)
57 | 7 | +64.900 | LAT = Latitude in decimal degrees
65 | 8 | +045.767 | LON = Longitude in decimal degrees
74 | 7 | +0071.0 | ELEV = Elevation in meters (altitude)
82 | 8 | 19590101 | BEGIN = Beginning Period Of Record (YYYYMMDD)
91 | 8 | 20140206 | END = Ending Period Of Record (YYYYMMDD)

## Téléchargement du dataset

le paragraphe suivant télécharge le dataset, et celui d'après définit une classe pour le *parsing* des données.
Comme indiqué précédemment, le fichier contient 22 lignes d'entête qu'il faut enlever. La version `RDD` de Spark ne sait pas enlever facilement ces lignes, alors on "triche" en utilisant une commande du *shell* linux:  

In [ ]:
!wget ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.txt -O input.txt
!tail -n +23 input.txt > isd-history.txt
!head isd-history.txt

In [ ]:
## classe Station
# chaque instance représente l'une des lignes du fichier isd-history.txt
class Station(object):

    # constructeur
    def __init__(self, ligne):
        self.ligne = ligne

    def __str__(self):
        return "Station[usaf=%s]" % self.usaf()

    def usaf(self):
        return self.ligne[0:0+6]

    def nom(self):
        return self.ligne[13:13+29].strip()

    def pays(self):
        return self.ligne[43:43+2].strip()

    def latitude(self):
        try:
            return float(self.ligne[57:57+7])
        except:
            return None

    def longitude(self):
        try:
            return float(self.ligne[65:65+8])
        except:
            return None

    def altitude(self):
        try:
            return float(self.ligne[74:74+7])
        except:
            return None

    def annee_debut(self):
        try:
            return int(self.ligne[82:82+4])
        except:
            return None

    def annee_fin(self):
        try:
            return int(self.ligne[91:91+4])
        except:
            return None

### Lecture des données


In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="TP2 RDD")

brut = sc.textFile("file:/home/jovyan/isd-history.txt")

## Exercice 1 : Répartition des stations par hémisphère

En se basant sur la latitude, il faudra compter le nombre de stations dans l’hémisphère nord et
dans l’hémisphère sud. Pour cela : 
* Il faut définir une fonction (isolée, pas une lambda) qui retourne "nord" ou "sud" selon la latitude. Vous devez ignorer les stations dont la latitude est inconnue.
* Il faut appeler cette fonction sur chaque station afin de construire des paires ("nord", 1) ou ("sud", 1)
* Il faut additionner les valeurs des paires ayant la même clé.

Utilisez ce que vous avez appris pour compter ces stations

In [ ]:
### 
###   VOTRE CODE ICI
###


## Exercice 2 : Plus grande période de mesures

On veut savoir quelle est l’identifiant USAF et le nom de la station qui a le plus grand écart
d’années entre le début de ses mesures et la fin. Par exemple, la station 225730 LESUKONSKOE
va de 1959 à 2014, ça fait 55 années mais ce n’est pas la plus longue période.

Pour cela : 
* L’année se trouve au début des champs BEGIN et END. Il suffit de la convertir en entier.
* Il faut d’abord garder les stations qui ont une année de début et une année de fin.
* Il faut construire des paires (fin-début, usaf+" "+nom)
* Il faut classer ces paires dans l’ordre des années d’écart décroissantes, puis garder la première.

Il y a aussi les méthodes `min` et `max` auxquelles on passe une fonction ou lambda qui permet
de spécifier sur quoi on recherche les extrêmes. L’exemple de la doc Spark n’est pas clair (usage de la
fonction str pour comparer les valeurs d’une liste). Voici par exemple comment trouver la station
la plus haute :


Bien comprendre que la lambda sert à sélectionner la valeur à maximiser, mais c’est le n-uplet
entier qui est retourné.
On peut aussi appliquer cette technique à une liste de paires (clé, valeur). Par exemple, retourner
la paire ayant la plus petite valeur :

C'est à vous : 

In [ ]:
### 
###   VOTRE CODE ICI
###



## Exercice 3 : Pays ayant le plus de stations

On veut savoir quel est le pays qui a le plus de stations. Le pays est un code sur deux lettres, ex:
FR. Il faut ignorer les pays incorrects.

A un moment, vous pourrez avoir besoin d’une astuce qui consiste à échanger les clés et les valeurs
dans les paires à traiter. Il suffit d’une lambda `(c,v): (v,c)`. Ou alors utiliser `sortBy` avec une
lambda qui extrait la valeur.

À vous de jouer : 

In [ ]:
### 
###   VOTRE CODE ICI
###


### Exercice 4 : Nombre de pays ayant des stations

On veut savoir combien de pays possèdent des stations météo. Pour cela, il faut faire la liste des
pays, mais en un seul exemplaire chacun, puis les compter.
Vous pourrez employer la méthode `distinct` ou trouver un moyen avec des paires (clé, valeur).
Il y a actuellement 249 pays listés (vous pouvez les trouver sur [cette page](https://www.iso.org/obp/ui/fr/#search)).

In [ ]:
### 
###   VOTRE CODE ICI
###


# Comment rendre ce TP

Dans le champ ci-dessous, indiquez votre nom et prénom.
Vous pouvez travailler en binômes, dans ce cas il faut indiquer les noms des deux collègues.
Ensuite : 
* exportez ce notebook (onglet File->Download as->Notebook(.ipynb)
* envoyer le fichier par email à [angelo.steffenel@univ-reims.fr](mailto:angelo.steffenel²univ-reims.fr)

## Date limite : vendredi 13/11/2020 (23h59)